# Sphere Manifold

Optimization on a sphere manifold, taken from [this Manopt example](https://www.manopt.org/manifold_documentation_sphere.html)


## Problem Description

Rayleigh quotient problem:
$$\min_{x}-x^TAx,$$
$$\text{s.t. }x^Tx=1,$$
where $A\in R^{n\times n}$ is a symmetric matrix. 

## Modules Importing
Import all necessary modules and add PyGRANSO src folder to system path.

In [1]:
import time
import torch
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct

## Data Initialization 
Specify torch device, and generate data

In [2]:
device = torch.device('cpu')
torch.manual_seed(0)
n = 300
# All the user-provided data (vector/matrix/tensor) must be in torch tensor format. 
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
A = torch.randn((n,n)).to(device=device, dtype=torch.double)
A = .5*(A+A.T)

## Function Set-Up

Encode the optimization variables, and objective and constraint functions.

Note: please strictly follow the format of comb_fn, which will be used in the PyGRANSO main algortihm.

In [3]:
# variables and corresponding dimensions.
var_in = {"x": [n,1]}


def user_fn(X_struct,A):
    x = X_struct.x
    
    # objective function
    f = -x.T@A@x

    # inequality constraint, matrix form
    ci = None
    
    # equality constraint 
    ce = pygransoStruct()
    ce.c1 = x.T@x-1

    return [f,ci,ce]

comb_fn = lambda X_struct : user_fn(X_struct,A)

## User Options
Specify user-defined options for PyGRANSO

In [4]:
opts = pygransoStruct()
opts.torch_device = device
opts.print_frequency = 10
opts.x0 = torch.randn((n,1)).to(device=device, dtype=torch.double)
opts.mu0 = 0.1 # increase penalty contribution
opts.opt_tol = 1e-6

## Main Algorithm

In [5]:
start = time.time()
soln = pygranso(var_spec = var_in,combined_fn = comb_fn,user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.0.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021 Tim Mitchell and Buyun Liang                                       ║ 
═════════════════════════════════════════════